# Collecte et Préparation des Données
Ce notebook automatise la récupération d'indicateurs de santé et socio-économiques auprès de trois sources majeures (OMS, Banque Mondiale, OpenStreetMap) pour 25 pays cibles. Il fusionne ensuite ces données dans un fichier unique appelé MASTER_DATASET.csv.

## 1. Importations et Configuration
Dans cette cellule, nous chargeons les bibliothèques nécessaires à la communication avec les API et à la manipulation des données.

In [1]:
import pandas as pd
import requests
import wbgapi as wb  # API de la Banque Mondiale
import osmnx as ox   # API OpenStreetMap
from time import sleep
import warnings
import os
import glob

# Désactivation des avertissements pour une lecture plus propre du notebook
warnings.filterwarnings('ignore')

print("Bibliothèques chargées avec succès.")

Bibliothèques chargées avec succès.


## 2. Définition de l'Échantillon des Pays
Nous sélectionnons 25 pays représentatifs de différents niveaux de revenus pour notre analyse.

In [2]:
countries = {
    # Revenus Faibles (Low income)
    'AFG': 'Afghanistan', 'RWA': 'Rwanda', 'UGA': 'Uganda', 'MDG': 'Madagascar',
    
    # Revenus Moyens-Inférieurs (Low middle income)
    'KEN': 'Kenya', 'BGD': 'Bangladesh', 'IND': 'India', 'PHL': 'Philippines', 
    'EGY': 'Egypt', 'PAK': 'Pakistan', 'NGA': 'Nigeria', 'GHA': 'Ghana',
    
    # Revenus Moyens-Supérieurs (High middle income)
    'BRA': 'Brazil', 'ZAF': 'South Africa', 'THA': 'Thailand', 'CHN': 'China', 
    'MEX': 'Mexico', 'TUR': 'Turkey', 'COL': 'Colombia', 'IDN': 'Indonesia',
    
    # Revenus Élevés (High income)
    'JPN': 'Japan', 'KOR': 'South Korea', 'SGP': 'Singapore', 'CHL': 'Chile', 'POL': 'Poland',
}

country_codes = list(countries.keys())
country_names = countries

print(f"Total des pays sélectionnés : {len(country_codes)}")

Total des pays sélectionnés : 25


## 3. Classe de Collecte de Données
Cette classe contient toutes les méthodes pour interroger les API internationales, sauvegarder les fichiers bruts et sélectionner des indicateurs pertinents à analyser. 

In [3]:
class HealthcareDataCollector:
    def __init__(self, countries=None, country_names_dict=None):
        """Initialisation avec les codes ISO3 et les noms complets des pays"""
        if countries is None:
            self.countries = country_codes
            self.country_names_dict = country_names
        else:
            self.countries = countries
            self.country_names_dict = country_names_dict if country_names_dict else {}
        
        self.who_data = {}
        self.wb_data = {}
        self.osm_data = {}
        
        print(f"\nCollecteur prêt pour {len(self.countries)} pays.")

    def collect_who_data(self):
        """Récupération des données médicales de l'OMS (GHO API)"""
        indicators = {
            'hospital_beds': 'WHS4_100',
            'doctors': 'HWF_0001',
            'nurses': 'WHS4_544',
            'dtp3_vaccination': 'MDG_0000000007',
            'tb_incidence': 'WHS6_102',
            'infant_mortality': 'MDG_0000000001',
            'maternal_mortality': 'MDG_0000000003',
        }

        for name, code in indicators.items():
            try:
                url = f"https://ghoapi.azureedge.net/api/{code}"
                response = requests.get(url, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    if 'value' in data:
                        df = pd.DataFrame(data['value'])
                        # Filtrage pour nos pays cibles
                        df = df[df['SpatialDim'].isin(self.countries)]
                        # Filtrage pour les années récentes (>=2010)
                        if 'TimeDim' in df.columns:
                            df['TimeDim'] = pd.to_numeric(df['TimeDim'], errors='coerce')
                            df = df[df['TimeDim'] >= 2010]
                        
                        self.who_data[name] = df
                        print(f"{name}: {len(df)} lignes ({df['SpatialDim'].nunique()} pays)")
                    else:
                        print(f"ERREUR : {name}: Pas de clé 'value' dans la réponse")
                else:
                    print(f"ERREUR : {name}: Code HTTP {response.status_code}")
                
                sleep(1) # Respect de l'API
            except Exception as e:
                print(f"ERREUR : {name}: {e}")

        print(f"\nCollecte OMS terminée : {len(self.who_data)} indicateurs récupérés")
        return self.who_data

    def collect_wb_data(self):
        """Récupération des données socio-économiques de la Banque Mondiale"""
        indicators = {
            'health_exp_gdp': 'SH.XPD.CHEX.GD.ZS',
            'health_exp_capita': 'SH.XPD.CHEX.PC.CD',
            'uhc_coverage': 'SH.UHC.SRVS.CV.XD',
            'population_total': 'SP.POP.TOTL',
            'population_density': 'EN.POP.DNST',
            'urban_population': 'SP.URB.TOTL.IN.ZS',
            'gdp_per_capita': 'NY.GDP.PCAP.CD',
            'poverty_headcount': 'SI.POV.DDAY',
            'gini_index': 'SI.POV.GINI',
        }

        for name, code in indicators.items():
            try:
                df = wb.data.DataFrame(
                    code, 
                    economy=self.countries,
                    time=range(2010, 2024),
                    labels=True,
                    skipBlanks=True,
                    columns='series'
                )
                self.wb_data[name] = df
                print(f"{name}: {df.shape[0]} pays - {df.shape[1]} années")
            except Exception as e:
                print(f"ERREUR : {name}: {e}")

        print(f"\nCollecte Banque Mondiale terminée : {len(self.wb_data)} indicateurs récupérés")
        return self.wb_data

    def collect_osm_data(self, country_names=None, batch_size=5):
        """Récupération des infrastructures via OpenStreetMap par lots"""
        if country_names is None:
            country_names = self.country_names_dict
        
        facility_types = ['hospital', 'clinic', 'pharmacy']
        countries_processed = 0
        
        for iso3, country_name in country_names.items():
            if iso3 not in self.countries:
                continue
                
            print(f"\n[{countries_processed + 1}/{len(self.countries)}] Traitement de {country_name} ({iso3})...")
            self.osm_data[iso3] = {}
            
            for facility_type in facility_types:
                try:
                    tags = {'amenity': facility_type}
                    facilities = ox.features_from_place(country_name, tags)
                    self.osm_data[iso3][facility_type] = facilities
                    print(f"  {facility_type}: {len(facilities)} structures")
                    sleep(2) 
                except Exception as e:
                    error_msg = str(e)
                    if 'Found no' in error_msg or 'Nominatim' in error_msg:
                        print(f"  ERREUR : {facility_type}: Pas de données ou erreur de localisation")
                    else:
                        print(f"  ERREUR : {facility_type}: {error_msg[:100]}")
                    self.osm_data[iso3][facility_type] = None
                    sleep(1)
            
            countries_processed += 1
            if countries_processed % batch_size == 0 and countries_processed < len(self.countries):
                print("Pause de 10s pour l'API...")
                sleep(10)

        print(f"\n✓ Collecte OSM terminée.")
        return self.osm_data

    def save_all_data(self, output_dir='healthcare_data'):
        """Sauvegarde de toutes les données collectées dans des fichiers locaux"""
        os.makedirs(output_dir, exist_ok=True)

        # Sauvegarde WHO
        who_dir = os.path.join(output_dir, 'who')
        os.makedirs(who_dir, exist_ok=True)
        for name, df in self.who_data.items():
            filepath = os.path.join(who_dir, f"{name}.csv")
            df.to_csv(filepath, index=False)
            print(f"Sauvegardé : who/{name}.csv")

        # Sauvegarde WB
        wb_dir = os.path.join(output_dir, 'worldbank')
        os.makedirs(wb_dir, exist_ok=True)
        for name, df in self.wb_data.items():
            filepath = os.path.join(wb_dir, f"{name}.csv")
            df.to_csv(filepath)
            print(f"Sauvegardé : worldbank/{name}.csv")

        # Sauvegarde OSM
        osm_dir = os.path.join(output_dir, 'osm')
        os.makedirs(osm_dir, exist_ok=True)
        osm_files_saved = 0
        for country, facilities in self.osm_data.items():
            for facility_type, gdf in facilities.items():
                if gdf is not None and len(gdf) > 0:
                    filepath = os.path.join(osm_dir, f"{country}_{facility_type}.geojson")
                    try:
                        gdf.to_file(filepath, driver='GeoJSON')
                        osm_files_saved += 1
                    except Exception as e:
                        print(f"ERREUR sauvegarde osm/{country}_{facility_type}.geojson: {e}")

        print(f"\nRésumé : OMS ({len(self.who_data)} fichiers), WB ({len(self.wb_data)} fichiers), OSM ({osm_files_saved} fichiers).")

    def get_summary_statistics(self):
        """Affichage des statistiques de couverture des données"""
        print(f"Données OMS :")
        for name, df in self.who_data.items():
            countries_with_data = df['SpatialDim'].nunique()
            print(f"  {name}: {countries_with_data}/{len(self.countries)} pays")
        
        print(f"Données Banque Mondiale :")
        for name, df in self.wb_data.items():
            non_null_count = df.notna().sum().sum()
            total_cells = df.shape[0] * df.shape[1]
            coverage = (non_null_count / total_cells * 100) if total_cells > 0 else 0
            print(f"  {name}: {coverage:.1f}% de couverture")

## 4. Exécution de la Collecte
Cette étape lance le processus de téléchargement et sauvegarde les résultats dans le dossier healthcare_data_25countries

In [4]:
# Initialisation du collecteur
collector = HealthcareDataCollector(
    countries=country_codes,
    country_names_dict=country_names
)

# Collecte des données OMS
print("--- Collecte OMS ---")
collector.collect_who_data()

# Collecte des données Banque Mondiale
print("\n--- Collecte Banque Mondiale ---")
collector.collect_wb_data()

# Collecte OSM (Commenté car long, à décommenter si besoin)
# collector.collect_osm_data(country_names)

# Statistiques et Sauvegarde
collector.get_summary_statistics()
collector.save_all_data('healthcare_data_25countries')

print("\nCollecte et sauvegarde terminées.")


Collecteur prêt pour 25 pays.
--- Collecte OMS ---
hospital_beds: 375 lignes (25 pays)
doctors: 263 lignes (25 pays)
nurses: 375 lignes (25 pays)
dtp3_vaccination: 2310 lignes (25 pays)
tb_incidence: 261 lignes (24 pays)
infant_mortality: 1050 lignes (25 pays)
maternal_mortality: 499 lignes (25 pays)

Collecte OMS terminée : 7 indicateurs récupérés

--- Collecte Banque Mondiale ---
health_exp_gdp: 330 pays - 3 années
health_exp_capita: 330 pays - 3 années
ERREUR : uhc_coverage: APIError: JSON decoding error (https://api.worldbank.org/v2/en/sources/2/series/SH.UHC.SRVS.CV.XD/country/AFG;RWA;UGA;MDG;KEN;BGD;IND;PHL;EGY;PAK;NGA;GHA;BRA;ZAF;THA;CHN;MEX;TUR;COL;IDN;JPN;KOR;SGP;CHL;POL/time/YR2010;YR2011;YR2012;YR2013;YR2014;YR2015;YR2016;YR2017;YR2018;YR2019;YR2020;YR2021;YR2022;YR2023?per_page=1000&page=1&format=json)
population_total: 350 pays - 3 années
population_density: 350 pays - 3 années
urban_population: 350 pays - 3 années
gdp_per_capita: 350 pays - 3 années
poverty_headcount: 17

## 5. Fonctions de Nettoyage et Fusion
Dans cette section, nous définissons les outils pour lire les fichiers enregistrés, uniformiser les formats (Pays, Année) et fusionner le tout.

In [5]:
def load_and_clean_who(filepath, indicator):
    """Lecture et nettoyage des données OMS (3 colonnes : Country, Year, Value)"""
    df = pd.read_csv(filepath)
    df = df.rename(columns={'SpatialDim': 'Country', 'TimeDim': 'Year', 'NumericValue' : indicator})
    df = df[['Country', 'Year', indicator]]
    
    # Gestion des doublons : on prend la moyenne si plusieurs entrées existent pour un pays/an
    df = df.groupby(['Country', 'Year'], as_index=False)[[indicator]].mean()
    return df

def load_and_clean_wb(filepath, indicator):
    """Lecture et nettoyage des données Banque Mondiale"""
    df = pd.read_csv(filepath)
    
    # On retire la colonne 'Country' originale pour garder 'economy' (qui correspond à l'ID ISO)
    if 'Country' in df.columns:
        df = df.drop(columns=['Country'])

    df.rename(columns={'economy': 'Country', 'Time': 'Year', df.columns[-1]: indicator}, inplace=True)
    df = df[['Country', 'Year', indicator]]
    return df

## 6. Création du Master Dataset
C'est l'étape finale où tous les indicateurs sont réunis dans un seul tableau.

In [6]:
# 1. Correspondance fichiers -> noms des colonnes indicateurs
who_files = {
    'doctors': 'doctors_per_10k',
    'hospital_beds': 'hospital_beds_per_10k',
    'infant_mortality': 'infant_mortality_rate',
    'maternal_mortality': 'maternal_mortality_rate',
    'tb_incidence': 'tb_incidence_per_100k',
    'dtp3_vaccination': 'dtp3_vaccination_rate',
    'nurses': 'nurses_per_10k',
}

wb_files = {
    'gdp_per_capita': 'gdp_per_capita',
    'population_density': 'pop_density',
    'health_exp_capita': 'health_exp_usd',
    'health_exp_gdp': 'health_exp_gdp',
    'population_total': 'population_total',
    'urban_population': 'urban_population',
    'poverty_headcount': 'poverty_headcount',
    'gini_index': 'gini_index',
}

# 2. Chargement et fusion progressive
print("--- Génération du Master Dataset ---")
base_path_who = 'healthcare_data_25countries/who'
base_path_wb = 'healthcare_data_25countries/worldbank'
df_to_merge = []

# Traitement OMS
for filename, col_name in who_files.items():
    path = os.path.join(base_path_who, f"{filename}.csv")
    if os.path.exists(path):
        clean_df = load_and_clean_who(path, col_name)
        df_to_merge.append(clean_df)

# Traitement Banque Mondiale
for filename, col_name in wb_files.items():
    path = os.path.join(base_path_wb, f"{filename}.csv")
    if os.path.exists(path):
        clean_df = load_and_clean_wb(path, col_name)
        df_to_merge.append(clean_df)

# Fusion (Outer join pour ne perdre aucune donnée même si certaines années manquent)
if df_to_merge:
    master_df = df_to_merge[0]
    for i in range(1, len(df_to_merge)):
        master_df = pd.merge(master_df, df_to_merge[i], on=['Country','Year'], how='outer')

# 3. Sauvegarde finale
master_df.to_csv('healthcare_data_25countries/MASTER_DATASET.csv', index=False)
print("Fichier final sauvegardé : healthcare_data_25countries/MASTER_DATASET.csv")
master_df.head()

--- Génération du Master Dataset ---
Fichier final sauvegardé : healthcare_data_25countries/MASTER_DATASET.csv


,Country,Year,doctors_per_10k,hospital_beds_per_10k,infant_mortality_rate,maternal_mortality_rate,tb_incidence_per_100k,dtp3_vaccination_rate,nurses_per_10k,gdp_per_capita,pop_density,health_exp_usd,health_exp_gdp,population_total,urban_population,poverty_headcount,gini_index
0,AFG,2010,2.44,66.0,76.860262,NaN,NaN,88.286126,66.0,560.621505,43.365207,46.424389,8.569672,28284089.0,22.261480,NaN,NaN
1,AFG,2011,2.58,68.0,73.942387,54.848922,4.459632,84.608192,68.0,606.694676,44.995949,52.187218,8.561908,29347708.0,22.820569,NaN,NaN
2,AFG,2012,2.46,67.0,71.194371,NaN,4.638738,81.184752,67.0,651.417134,46.854689,52.452473,7.897169,30560034.0,23.343439,NaN,NaN
3,AFG,2013,2.90,64.0,68.625671,43.840327,4.596064,77.996380,64.0,637.087099,48.483977,56.160435,8.805964,31622704.0,23.807714,NaN,NaN
4,AFG,2014,3.04,62.0,66.279219,38.722645,4.558965,75.091706,63.0,625.054942,50.277545,60.049511,9.528878,32792523.0,24.191017,NaN,NaN


## 7. Indicateurs sélectionnés

Nous avons sélectionné quinze indicateurs médicaux ou socio-économiques dans les bases de l'OMS et de la Banque Mondiale : 

Indicateurs de l'OMS : 

* `doctors_per_10k` : nombre de médecins pour 10 000 habitants. Indicateur de densité du personnel médical.

* `hospital_beds_per_10k` : nombre de lits d'hôpitaux pour 10 000 habitants.

* `infant_mortality_rate` : taux de mortalité infantile : combien d’enfants sur 1000 nés vivants sont décédés avant leur premier anniversaire.

* `maternal_mortality_rate` : mortalité marternelle : nombre de décès de mères dus à des causes liées à la grossesse ou à l’accouchement pour 100 000 naissances vivantes.

* `tb_incidence_per_100k` : taux d'incidence de la tuberculose : le nombre estimé de nouveaux cas de tuberculose et de rechutes survenant au cours d'une année donnée, exprimé sous forme de taux pour 100 000 habitants.

* `dtp3_vaccination` : couverture vaccinale DTP3 (DTC3 en français) chez les enfants d'un an : le pourcentage d'enfants d'un an ayant reçu trois doses du vaccin combiné contre la diphtérie, le tétanos et la coqueluche au cours d'une année donnée.

* `nurses_per_10k` : nombre de personnels infirmiers et de sages-femmes pour 10 000 habitants.

Indicateurs de la Banque Mondiale : 

* `gdp_per_capita` : PIB par habitant.

* `pop_density` : densité de population.

* `health_exp_usd` : niveau des dépenses de santé courantes par personne, mesuré en dollars américains courants.

* `health_exp_gdp` : niveau des dépenses totales consacrées à la santé (publique + privée) en proportion du PIB.

* `population_total` : population totale.

* `urban_population` : population urbaine ( en % de la population totale).

* `poverty_headcount` : taux d’incidence de la pauvreté (en % de la population totale) : pourcentage de personnes vivant en dessous d’une ligne de pauvreté internationale.

* `gini_index` : indice de Gini.

